## **Crawling을 통해 주가&시가총액 데이터 수집**

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import datetime
import os
import pandas as pd
import glob

START_DATE = '20220101'
END_DATE = '20220110'
DOWNLOAD_DIR = r"C:\Users\YunJi\Desktop\Projects\dart\krx_downloads"

options = Options()
options.add_experimental_option('prefs', {'download.default_directory': DOWNLOAD_DIR})
service = Service('chromedriver.exe')

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

In [ ]:
date_list = []
start = datetime.datetime.strptime(START_DATE, "%Y%m%d")
end = datetime.datetime.strptime(END_DATE, "%Y%m%d")
delta = datetime.timedelta(days=1)
while start <= end:
    if start.weekday() < 5:  # 평일만
        date_list.append(start.strftime("%Y%m%d"))
    start += delta

In [ ]:
url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101'
driver.get(url)
time.sleep(5)

In [ ]:
for date in date_list:
    try:
        print(f"{date} 크롤링 시도중...")
        # (1) 값 할당 + change 트리거
        driver.execute_script(f'document.getElementById("trdDd").value="{date}";')
        driver.execute_script('$("#trdDd").trigger("change");')
        time.sleep(0.2)
        # (2) send_keys까지 중복 적용
        date_input = driver.find_element(By.ID, 'trdDd')
        #date_input.clear()
        date_input.send_keys(date)
        date_input.send_keys(Keys.TAB)
        time.sleep(0.3)

        search_btn = driver.find_element(By.ID, 'jsSearchButton')
        search_btn.click()
        time.sleep(2.5)

        excel1_btn = driver.find_element(By.CLASS_NAME, 'CI-MDI-UNIT-DOWNLOAD')
        excel1_btn.click()
        excel2_btn = driver.find_element(By.XPATH, '//div[@data-type="csv"]/a')
        excel2_btn.click()
        time.sleep(3.5)

        print(f"  {date} 완료")
        latest_file = max([os.path.join(DOWNLOAD_DIR, f) for f in os.listdir(DOWNLOAD_DIR)], key=os.path.getctime)
        new_file = os.path.join(DOWNLOAD_DIR, f'krx_{date}.csv')
        os.rename(latest_file, new_file)

        date_input.clear()

    except Exception as e:
        print(f"  {date} 실패: {e}")


## **주가&시가총액 데이터 가공**

In [ ]:
path = "C:/Users/YunJi/Desktop/Projects/dart/krx_downloads"  
all_files = glob.glob(os.path.join(path, "*.csv"))

result = None
market_info = {}

for file in sorted(all_files):  
    basename = os.path.basename(file)
    date_str = basename.split('_')[1].split('.')[0][2:]  

    df = pd.read_csv(file, encoding='cp949')

    for idx, row in df.iterrows():
        code = row['종목코드']
        market = row['시장구분']

        if code in market_info and market_info[code] != market:
            print(f'경고: {code} {row["종목명"]} 시장구분 변경: {market_info[code]} → {market}')
        
        market_info[code] = market 

    df_part = df[['종목코드', '종목명', '종가', '시가총액']].copy()
    df_part.rename(columns={
        '종가': f'{date_str}종가',
        '시가총액': f'{date_str}시가총액'
    }, inplace=True)

    if result is None:
        result = df_part
    else:
        result = pd.merge(result, df_part, on=['종목코드', '종목명'], how='outer')

market_df = pd.DataFrame(list(market_info.items()), columns=['종목코드', '시장구분'])
final_result = pd.merge(result, market_df, on='종목코드', how='left')
cols = ['시장구분', '종목코드', '종목명'] + [col for col in final_result.columns if col not in ['시장구분', '종목코드', '종목명']]
final_result = final_result[cols]

folder_path = "C:/Users/YunJi/Desktop/Projects/dart"
file_name = "주가.csv"
full_path = os.path.join(folder_path, file_name)
final_result.to_csv(full_path, index=False, encoding='utf-8-sig')

## **산업군 데이터와 merge**

In [ ]:
df1 = final_result
df2 = pd.read_csv("C:/Users/YunJi/Desktop/Projects/dart/crawling_주가_raw.csv", encoding='cp949')[['종목명', '업종명']]

merged = pd.merge(df1, df2, on='종목명', how='left')
merged.to_csv("full.csv")

## **유상증자, 무상증자 기업에 주가 데이터 merge**

In [ ]:
df1 = pd.read_csv("C:/Users/YunJi/Desktop/Projects/dart/full.csv") 
df2 = pd.read_excel("C:/Users/YunJi/Desktop/Projects/dart/opendart_무상증자_raw.xls")  
df3 = pd.read_excel("C:/Users/YunJi/Desktop/Projects/dart/opendart_유상증자_raw.xls")   

df2_reduced = df2[["회사명", "신주배정기준일", "1주당 신주배정 주식수(보통주식 (주))"]].copy()
merged = pd.merge(df1, df2_reduced, on="회사명", how="left")

df3_reduced = df3[["회사명", "접수일", "신주의 종류와 수(보통주식 (주))", "자금조달의 목적(시설자금 (원))", "자금조달의 목적(영업양수자금 (원))", "자금조달의 목적(운영자금 (원))", "자금조달의 목적(채무상환자금 (원))", "자금조달의 목적(타법인 증권 취득자금 (원))", "자금조달의 목적(기타자금 (원))", "증자방식"]].copy()
merged2 = pd.merge(merged, df2_reduced, on="회사명", how="left")

merged2.to_excel("event.xlsx", index=False)

## **유상증자, 무상증자 발생회사의 1개월, 6개월 수익률 계산**

In [ ]:
import pandas as pd

# 📌 1. 엑셀 파일 불러오기 + 컬럼 정리
file_path = "/Users/yebin/Downloads/merged_ver4.xlsx"  # ← 경로는 네 환경에 맞게 수정
df = pd.read_excel(file_path)

# 컬럼명 공백 제거
df.columns = df.columns.str.strip()

# 📅 2. 종가 열 파악 및 날짜 맵핑
price_cols = [col for col in df.columns if "종가" in col]
date_map = {col: pd.to_datetime(col.replace("종가", "")) for col in price_cols}
date_cols_sorted = sorted(date_map.items(), key=lambda x: x[1])

# ⚙️ 3. 수익률 계산 함수
def get_returns_by_event(row, event_col):
    from pandas import DateOffset
    if pd.isna(row[event_col]):
        return None
    try:
        event_date = pd.to_datetime(row[event_col])
    except:
        return None

    result = {
        "회사명": row["회사명"],
        "이벤트일": event_date.strftime('%Y-%m-%d'),
        "이벤트종류": "무상증자" if event_col == "무상증자 신주배정기준일" else "유상증자"
    }

    base_col = next((col for col, d in date_cols_sorted if d >= event_date and not pd.isna(row[col])), None)
    if not base_col:
        return None
    base_price = row[base_col]

    for m in [1, 6]:
        target_date = event_date + DateOffset(months=m)
        target_col = next((col for col, d in date_cols_sorted if d >= target_date and not pd.isna(row[col])), None)
        if target_col:
            return_rate = (row[target_col] - base_price) / base_price
            result[f"{m}개월 수익률"] = return_rate
        else:
            result[f"{m}개월 수익률"] = None

    return result

# 🔁 4. 전체 행에 수익률 계산 적용
results = []
for _, row in df.iterrows():
    for event_col in ["무상증자 신주배정기준일", "유상증자 접수일"]:
        r = get_returns_by_event(row, event_col)
        if r:
            results.append(r)

# 📊 5. 이벤트별 수익률 결과 정리
event_returns_df = pd.DataFrame(results)

print(event_returns_df.shape)
print(event_returns_df.head())
print(len(results))  # 수익률 계산된 row 수

# 💾 6. 이벤트별 수익률 엑셀 저장
event_returns_df.to_excel("이벤트별_수익률_결과.xlsx", index=False)

# 📈 7. 기업별 평균 수익률 계산
event_returns_df.columns = [str(col).strip() for col in event_returns_df.columns]  # 혹시 모르니 다시 정리
average_returns = (
    event_returns_df
    .groupby("회사명")[["1개월 수익률", "6개월 수익률"]]
    .mean()
    .reset_index()
)

# 💾 8. 기업별 평균 수익률 엑셀 저장
average_returns.to_excel("기업별_평균_수익률_결과.xlsx", index=False)


In [ ]:
import pandas as pd
from pandas import DateOffset

# 📌 1. 데이터 불러오기
file_path = "/Users/yebin/Downloads/merged_ver4.xlsx"
df = pd.read_excel(file_path)
df.columns = df.columns.str.strip()

# 📅 2. 종가 컬럼 정리
price_cols = [col for col in df.columns if "종가" in col]

# 종가 날짜 파싱: YYMMDD → 20YYMMDD 변환
date_map = {
    col: pd.to_datetime("20" + col.replace("종가", ""), format="%Y%m%d", errors="coerce")
    for col in price_cols
}

# 날짜순 정렬
date_cols_sorted = sorted(date_map.items(), key=lambda x: x[1])

from pandas import DateOffset

def get_returns_by_event(row):
    company = row["회사명"]
    event_date = parse_korean_date(row["무상증자 신주배정기준일"])

    if pd.isna(event_date):
        print(f"❌ [{company}] 날짜 변환 실패")
        return None

    base_col = next((col for col, d in date_cols_sorted if d >= event_date and not pd.isna(row.get(col))), None)
    if not base_col:
        print(f"❌ [{company}] 기준일 이후 종가 없음")
        return None

    base_price = row[base_col]
    result = {
        "회사명": company,
        "이벤트일": event_date.strftime('%Y-%m-%d'),
        "이벤트종류": "무상증자"
    }

    success = False
    for m in [1, 6]:
        target_date = event_date + DateOffset(months=m)
        target_col = next((col for col, d in date_cols_sorted if d >= target_date and not pd.isna(row.get(col))), None)
        if target_col:
            return_rate = (row[target_col] - base_price) / base_price
            result[f"{m}개월 수익률"] = return_rate
            success = True
        else:
            result[f"{m}개월 수익률"] = None

    return result if success else None

df_free = df[
    df["무상증자 신주배정기준일"].notna() &
    df["회사명"].notna() &
    (df["회사명"].astype(str).str.strip() != "")
]

results = []
for _, row in df_free.iterrows():
    r = get_returns_by_event(row)
    if r:
        results.append(r)

free_returns_df = pd.DataFrame(results)
print("결과 개수:", len(free_returns_df))
print(free_returns_df.head())

print(df_free.shape)
print(df_free[["회사명", "무상증자 신주배정기준일"]].head())

print("🧾 전체 컬럼명:")
for col in free_returns_df.columns:
    print(f"'{col}'")

print(free_returns_df.columns.tolist())  # 리스트로 전체 컬럼 확인

# 📊 6. 결과 저장
free_returns_df = pd.DataFrame(results)
free_returns_df.to_excel("무상증자_수익률_결과.xlsx", index=False)

# 📈 7. 기업별 평균 수익률 계산
avg_free = (
    free_returns_df
    .groupby("회사명")[["1개월 수익률", "6개월 수익률"]]
    .mean()
    .reset_index()
)
avg_free.to_excel("무상증자_기업별_평균수익률.xlsx", index=False)

print("✅ 무상증자 기업만 추려 수익률 계산 및 저장 완료!")

In [ ]:
import pandas as pd

# 엑셀 파일 불러오기
file_path = "/mnt/data/유상증자_수익률_결과.xlsx"
df = pd.read_excel(file_path)

# 컬럼명 공백 제거
df.columns = df.columns.str.strip()

# 1. 1개월 수익률과 6개월 수익률이 모두 0인 행 제거
df_filtered = df[~((df["1개월 수익률"] == 0) & (df["6개월 수익률"] == 0))]

# 2. 결측치는 각 열의 평균값으로 대체
df_imputed = df_filtered.copy()
for col in ["1개월 수익률", "6개월 수익률"]:
    mean_val = df_imputed[col].mean()
    df_imputed[col].fillna(mean_val, inplace=True)

# 결과 확인
import ace_tools as tools; tools.display_dataframe_to_user(name="결측치 처리된 수익률 데이터", dataframe=df_imputed)

# 저장
df_imputed.to_excel("C:/Users/YunJi/Desktop/Projects/dart/event_return_rate_유상.xlsx", index=False)

In [ ]:
# 무상증자 수익률 파일 불러오기
file_path_bonus = "/mnt/data/무상증자_수익률_결과.xlsx"
df_bonus = pd.read_excel(file_path_bonus)

# 컬럼명 공백 제거
df_bonus.columns = df_bonus.columns.str.strip()

# 1. 1개월 수익률과 6개월 수익률이 모두 0인 행 제거
df_bonus_filtered = df_bonus[~((df_bonus["1개월 수익률"] == 0) & (df_bonus["6개월 수익률"] == 0))]

# 2. 결측치는 각 열의 평균값으로 대체
df_bonus_imputed = df_bonus_filtered.copy()
for col in ["1개월 수익률", "6개월 수익률"]:
    mean_val = df_bonus_imputed[col].mean()
    df_bonus_imputed[col].fillna(mean_val, inplace=True)

# 결과 확인
tools.display_dataframe_to_user(name="결측치 처리된 무상증자 수익률 데이터", dataframe=df_bonus_imputed)

# 저장
df_bonus_imputed.to_excel("C:/Users/YunJi/Desktop/Projects/dart/event_return_rate_무상.xlsx", index=False)

## **유상증자, 무상증자한 회사의 수익률과 같은 날짜, 산업군의 수익률이 같은지 비교**

In [ ]:
import pandas as pd
musang_rate = pd.read_excel("C:/Users/YunJi/Desktop/Projects/dart/event_return_rate_무상.xlsx")
yusang_rate = pd.read_excel("C:/Users/YunJi/Desktop/Projects/dart/event_return_rate_유상.xlsx")
industry_type_rate = pd.read_excel("C:/Users/YunJi/Desktop//Projects/dart/industry_type_return_rate.xlsx", sheet_name="보정_일별_업종수익률").drop('번호', axis=1)

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)
pd.set_option('display.max_rows', None)

indutype1 = pd.read_excel("C:/Users/YunJi/Desktop/Projects/dart/krx_업종분류_raw_2022.xlsx").rename(columns={'종목명': '회사명'})[['회사명', '업종명']]
indutype2 = pd.read_excel("C:/Users/YunJi/Desktop/Projects/dart/krx_업종분류_raw_2023.xlsx").rename(columns={'종목명': '회사명'})[['회사명', '업종명']]
indutype3 = pd.read_excel("C:/Users/YunJi/Desktop/Projects/dart/krx_업종분류_raw_2024.xlsx").rename(columns={'종목명': '회사명'})[['회사명', '업종명']]
indutype4 = pd.read_excel("C:/Users/YunJi/Desktop/Projects/dart/krx_업종분류_raw_2025.xlsx").rename(columns={'종목명': '회사명'})[['회사명', '업종명']]
indutype = pd.concat([indutype1, indutype2, indutype3, indutype4], ignore_index=True).drop_duplicates(subset='회사명', keep='last')

# new_indutype = pd.read_excel("C:/Users/YunJi/Desktop/final_merged1.xlsx")[['회사명', '업종명']].drop_duplicates(subset='회사명', keep='last')

industry_type_rate = industry_type_rate.rename(columns={'1개월_수익률':'동업_1개월수익률', '6개월_수익률':'동업_6개월수익률'})

In [ ]:
event_rate = pd.concat([musang_rate, yusang_rate], ignore_index=True)
event_rate = pd.merge(event_rate,indutype, on='회사명', how='left').drop_duplicates().sort_values(by='이벤트일')
event_rate = event_rate.rename(columns={'이벤트일': '날짜', '1개월 수익률':'1개월수익률', '6개월 수익률':'6개월수익률'})

In [ ]:
df = pd.merge(event_rate, industry_type_rate, on=['업종명', '날짜'], how='left')
df = df.dropna(subset=['업종명', '동업_1개월수익률'])
df = df[~((df['1개월수익률'].isna()) | (df['1개월수익률'] == 0) | (df['6개월수익률'].isna()) | (df['6개월수익률'] == 0))]


In [ ]:
df_yusang = (
    df.query("이벤트종류=='유상증자'").groupby('업종명', as_index=False).agg(
    event_1mean=('1개월수익률', 'mean'), 
    event_6mean=('6개월수익률', 'mean'), 
    field_1mean=('동업_1개월수익률', 'mean'), 
    field_6mean=('동업_6개월수익률', 'mean'),
    event_1var=('1개월수익률', 'var'),
    event_6var=('6개월수익률', 'var'),
    field_1var=('동업_1개월수익률', 'var'),
    field_6var=('동업_6개월수익률', 'var')
    )
)

df_musang = (
    df.query("이벤트종류=='무상증자'").groupby('업종명', as_index=False).agg(
    event_1mean=('1개월수익률', 'mean'), 
    event_6mean=('6개월수익률', 'mean'), 
    field_1mean=('동업_1개월수익률', 'mean'), 
    field_6mean=('동업_6개월수익률', 'mean'),
    event_1var=('1개월수익률', 'var'),
    event_6var=('6개월수익률', 'var'),
    field_1var=('동업_1개월수익률', 'var'),
    field_6var=('동업_6개월수익률', 'var')
    )
)

count_yusang = df.query("이벤트종류=='유상증자'").groupby('업종명', as_index=False).size().rename(columns={'size': '건수'})
count_musang = df.query("이벤트종류=='무상증자'").groupby('업종명', as_index=False).size().rename(columns={'size': '건수'})

df_yusang = df_yusang.merge(count_yusang, on='업종명', how='left')
df_musang = df_musang.merge(count_musang, on='업종명', how='left')

In [ ]:
df = df.query("업종명 in ['금속', '기계·장비', '운송장비·부품', '유통', '의료·정밀기기', '일반서비스', '전기·전자', '제약', '화학']").query("이벤트종류=='유상증자'")

In [ ]:
from scipy.stats import ttest_ind, levene
import numpy as np

# 결과 저장 리스트
results = []

# 그룹핑
grouped = df.groupby(['업종명'])

for (sector), group in grouped:
    # 1개월 수익률 비교
    x1_1m = group['1개월수익률'].dropna()
    x2_1m = group['동업_1개월수익률'].dropna()

    # 6개월 수익률 비교
    x1_6m = group['6개월수익률'].dropna()
    x2_6m = group['동업_6개월수익률'].dropna()

    ## 1개월 분석
    if len(x1_1m) > 1 and len(x2_1m) > 1:
        p_levene_1m = levene(x1_1m, x2_1m).pvalue
        equal_var_1m = p_levene_1m > 0.05
        t_1m, p_1m = ttest_ind(x1_1m, x2_1m, equal_var=equal_var_1m)
    else:
        p_levene_1m = t_1m = p_1m = np.nan
        equal_var_1m = np.nan

    ## 6개월 분석
    if len(x1_6m) > 1 and len(x2_6m) > 1:
        p_levene_6m = levene(x1_6m, x2_6m).pvalue
        equal_var_6m = p_levene_6m > 0.05
        t_6m, p_6m = ttest_ind(x1_6m, x2_6m, equal_var=equal_var_6m)
    else:
        p_levene_6m = t_6m = p_6m = np.nan
        equal_var_6m = np.nan

    results.append({
        '업종명': sector,
        'n_event': len(x1_1m),
        'n_field': len(x2_1m),
        '1개월_등분산_p': p_levene_1m,
        '1개월_등분산가정': equal_var_1m,
        '1개월_t값': t_1m,
        '1개월_p값': p_1m,
        '6개월_등분산_p': p_levene_6m,
        '6개월_등분산가정': equal_var_6m,
        '6개월_t값': t_6m,
        '6개월_p값': p_6m
    })

result_df = pd.DataFrame(results)
display(result_df)


In [ ]:
# 1개월 수익률에서 유의미한 그룹만 필터링
significant_1m = result_df[result_df['1개월_p값'] < 0.05]

# 6개월 수익률에서 유의미한 그룹만 필터링
significant_6m = result_df[result_df['6개월_p값'] < 0.05]

# 확인
print("1개월 유의미한 그룹:")
display(significant_1m)

print("6개월 유의미한 그룹:")
display(significant_6m)